In [2]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import openslide
from tqdm import tqdm
import cv2 as cv
from natsort import natsorted

thispath = Path.cwd().resolve()

image_index = "000030870900342376"

def available_magnifications(mpp, level_downsamples):
	mpp = float(mpp)
	if (mpp<0.26):
		magnification = 40
	else:
		magnification = 20
	
	mags = []
	for l in level_downsamples:
		mags.append(magnification/l)
	
	return mags

datadir = Path("/mnt/nas4/datasets/ToReadme/ExaMode_Dataset1/AOEC")
maskdir = Path(thispath.parent / "data" / "Mask_PyHIST")

svs_files = [i for i in datadir.rglob("*.svs") if "LungAOEC" in str(i) and image_index in str(i)]
svs_file = svs_files[0]

header = ["level_dimensions", "level_downsamples", "magnifications", "mpp",
              "number_patches_pyhist", "patch_shape", "center"]
metadata = pd.DataFrame(columns=header)

patchdir = Path(maskdir / svs_file.parent.stem / svs_file.stem / f"{svs_file.stem}_tiles")
    
number_patches = len(os.listdir(Path(maskdir / svs_file.parent.stem / svs_file.stem / f"{svs_file.stem}_tiles")))

patch = cv.imread(str(Path(patchdir / os.listdir(patchdir)[0] )))
patch_shape = patch.shape

center = svs_file.parent.stem.split("_")[0]

slide = openslide.OpenSlide(str(svs_file))

level_dimensions = slide.level_dimensions
mpp = slide.properties['openslide.mpp-x']
level_downsamples = slide.level_downsamples
mags = available_magnifications(mpp, level_downsamples)
metadata.loc[svs_file.stem] = [level_dimensions, level_downsamples, mags,
                               mpp, number_patches, patch_shape, center]

print(metadata)

                                                     level_dimensions  \
000030870900342376  ((147408, 82667), (36852, 20666), (9213, 5166)...   

                                                    level_downsamples  \
000030870900342376  (1.0, 4.000072582986547, 16.001064653503676, 6...   

                                                       magnifications  \
000030870900342376  [40.0, 9.999818545826253, 2.499833658958524, 0...   

                                    mpp  number_patches_pyhist    patch_shape  \
000030870900342376  0.25180000000000002                   5064  (224, 224, 3)   

                      center  
000030870900342376  LungAOEC  
